<a href="https://colab.research.google.com/github/jarl24-dev/stock-markets-analytics-zoomcamp/blob/main/03-modelling/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install yfinance

In [4]:
# read files shared via google-drive-link
# https://stackoverflow.com/questions/62759748/downloading-data-from-a-shared-google-drive-link-in-google-colab

!pip uninstall gdown -y && pip install gdown
!gdown -V

Found existing installation: gdown 5.2.0
Uninstalling gdown-5.2.0:
  Successfully uninstalled gdown-5.2.0
gdown 5.2.0 at /usr/local/lib/python3.11/dist-packages


In [5]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

# 0) Dataset for Modeling: Final Preparations

## 0.1) Importing data from Drive & defining variable sets
* automated version need to have a daily updated file/database entries

In [6]:
# https://stackoverflow.com/questions/62759748/downloading-data-from-a-shared-google-drive-link-in-google-colab
# truncated data from Module 2: https://drive.google.com/file/d/1mb0ae2M5AouSDlqcUnIwaHq7avwGNrmB/view?usp=sharing
!gdown https://drive.google.com/file/d/1mb0ae2M5AouSDlqcUnIwaHq7avwGNrmB/view?usp=sharing --fuzzy -O /content/


Downloading...
From (original): https://drive.google.com/uc?id=1mb0ae2M5AouSDlqcUnIwaHq7avwGNrmB
From (redirected): https://drive.google.com/uc?id=1mb0ae2M5AouSDlqcUnIwaHq7avwGNrmB&confirm=t&uuid=289a3591-c03a-4021-8002-16fa173646ca
To: /content/stocks_df_combined_2025_06_13.parquet.brotli
100% 130M/130M [00:00<00:00, 179MB/s]


In [7]:
# truncated
# df = pd.read_parquet("/content/stocks_df_combined_trunc_2014_2023.parquet.brotli", )

# full dataset for 33 stocks
df_full = pd.read_parquet("/content/stocks_df_combined_2025_06_13.parquet.brotli", )


In [8]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 230262 entries, 0 to 5700
Columns: 203 entries, Open to growth_btc_usd_365d
dtypes: datetime64[ns](3), float64(129), int32(64), int64(5), object(2)
memory usage: 302.2+ MB


In [9]:
df_full.keys()

Index(['Open', 'High', 'Low', 'Close_x', 'Volume', 'Dividends', 'Stock Splits',
       'Ticker', 'Year', 'Month',
       ...
       'growth_brent_oil_7d', 'growth_brent_oil_30d', 'growth_brent_oil_90d',
       'growth_brent_oil_365d', 'growth_btc_usd_1d', 'growth_btc_usd_3d',
       'growth_btc_usd_7d', 'growth_btc_usd_30d', 'growth_btc_usd_90d',
       'growth_btc_usd_365d'],
      dtype='object', length=203)

In [10]:
df_full[['Year','Month','Weekday']].dtypes

,0
Year,int32
Month,datetime64[ns]
Weekday,int32


In [11]:
for i in df_full.keys():
  print(i)

Open
High
Low
Close_x
Volume
Dividends
Stock Splits
Ticker
Year
Month
Weekday
Date
growth_1d
growth_3d
growth_7d
growth_30d
growth_90d
growth_365d
growth_future_30d
SMA10
SMA20
growing_moving_average
high_minus_low_relative
volatility
is_positive_growth_30d_future
ticker_type
index_x
adx
adxr
apo
aroon_1
aroon_2
aroonosc
bop
cci
cmo
dx
macd
macdsignal
macdhist
macd_ext
macdsignal_ext
macdhist_ext
macd_fix
macdsignal_fix
macdhist_fix
mfi
minus_di
mom
plus_di
dm
ppo
roc
rocp
rocr
rocr100
rsi
slowk
slowd
fastk
fastd
fastk_rsi
fastd_rsi
trix
ultosc
willr
index_y
ad
adosc
obv
atr
natr
ht_dcperiod
ht_dcphase
ht_phasor_inphase
ht_phasor_quadrature
ht_sine_sine
ht_sine_leadsine
ht_trendmod
avgprice
medprice
typprice
wclprice
index
cdl2crows
cdl3blackrows
cdl3inside
cdl3linestrike
cdl3outside
cdl3starsinsouth
cdl3whitesoldiers
cdlabandonedbaby
cdladvancedblock
cdlbelthold
cdlbreakaway
cdlclosingmarubozu
cdlconcealbabyswall
cdlcounterattack
cdldarkcloudcover
cdldoji
cdldojistar
cdldragonflydoji


In [12]:
# growth indicators (but not future growth)
GROWTH = [g for g in df_full.keys() if (g.find('growth_')==0)&(g.find('future')<0)]
GROWTH

['growth_1d',
 'growth_3d',
 'growth_7d',
 'growth_30d',
 'growth_90d',
 'growth_365d',
 'growth_dax_1d',
 'growth_dax_3d',
 'growth_dax_7d',
 'growth_dax_30d',
 'growth_dax_90d',
 'growth_dax_365d',
 'growth_snp500_1d',
 'growth_snp500_3d',
 'growth_snp500_7d',
 'growth_snp500_30d',
 'growth_snp500_90d',
 'growth_snp500_365d',
 'growth_dji_1d',
 'growth_dji_3d',
 'growth_dji_7d',
 'growth_dji_30d',
 'growth_dji_90d',
 'growth_dji_365d',
 'growth_epi_1d',
 'growth_epi_3d',
 'growth_epi_7d',
 'growth_epi_30d',
 'growth_epi_90d',
 'growth_epi_365d',
 'growth_gold_1d',
 'growth_gold_3d',
 'growth_gold_7d',
 'growth_gold_30d',
 'growth_gold_90d',
 'growth_gold_365d',
 'growth_wti_oil_1d',
 'growth_wti_oil_3d',
 'growth_wti_oil_7d',
 'growth_wti_oil_30d',
 'growth_wti_oil_90d',
 'growth_wti_oil_365d',
 'growth_brent_oil_1d',
 'growth_brent_oil_3d',
 'growth_brent_oil_7d',
 'growth_brent_oil_30d',
 'growth_brent_oil_90d',
 'growth_brent_oil_365d',
 'growth_btc_usd_1d',
 'growth_btc_usd_3d',


In [13]:
# leaving only Volume ==> generate ln(Volume)
OHLCV = ['Open','High','Low','Close','Adj Close_x','Volume']

In [14]:
CATEGORICAL = ['Month', 'Weekday', 'Ticker', 'ticker_type']

In [15]:
TO_PREDICT = [g for g in df_full.keys() if (g.find('future')>=0)]
TO_PREDICT

['growth_future_30d', 'is_positive_growth_30d_future']

In [16]:
TO_DROP = ['Year','Date','index_x', 'index_y', 'index', 'Quarter','Adj Close_y'] + CATEGORICAL + OHLCV
TO_DROP

['Year',
 'Date',
 'index_x',
 'index_y',
 'index',
 'Quarter',
 'Adj Close_y',
 'Month',
 'Weekday',
 'Ticker',
 'ticker_type',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close_x',
 'Volume']

In [17]:
# let's define on more custom numerical features
df_full['ln_volume'] = df_full.Volume.apply(lambda x: np.log(x))

/tmp/ipython-input-17-2841410488.py:2: RuntimeWarning: divide by zero encountered in log
  df_full['ln_volume'] = df_full.Volume.apply(lambda x: np.log(x))


In [18]:
# manually defined features
CUSTOM_NUMERICAL = ['SMA10', 'SMA20', 'growing_moving_average', 'high_minus_low_relative','volatility', 'ln_volume']

In [19]:
# All Supported Ta-lib indicators: https://github.com/TA-Lib/ta-lib-python/blob/master/docs/funcs.md

TECHNICAL_INDICATORS = ['adx', 'adxr', 'apo', 'aroon_1','aroon_2', 'aroonosc',
 'bop', 'cci', 'cmo','dx', 'macd', 'macdsignal', 'macdhist', 'macd_ext',
 'macdsignal_ext', 'macdhist_ext', 'macd_fix', 'macdsignal_fix',
 'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di', 'dm', 'ppo',
 'roc', 'rocp', 'rocr', 'rocr100', 'rsi', 'slowk', 'slowd', 'fastk',
 'fastd', 'fastk_rsi', 'fastd_rsi', 'trix', 'ultosc', 'willr',
 'ad', 'adosc', 'obv', 'atr', 'natr', 'ht_dcperiod', 'ht_dcphase',
 'ht_phasor_inphase', 'ht_phasor_quadrature', 'ht_sine_sine', 'ht_sine_leadsine',
 'ht_trendmod', 'avgprice', 'medprice', 'typprice', 'wclprice']

In [20]:
TECHNICAL_PATTERNS = [g for g in df_full.keys() if g.find('cdl')>=0]
print(f'Technical patterns count = {len(TECHNICAL_PATTERNS)}, examples = {TECHNICAL_PATTERNS[0:5]}')


Technical patterns count = 61, examples = ['cdl2crows', 'cdl3blackrows', 'cdl3inside', 'cdl3linestrike', 'cdl3outside']


In [21]:
MACRO = ['gdppot_us_yoy', 'gdppot_us_qoq', 'cpi_core_yoy', 'cpi_core_mom', 'FEDFUNDS',
 'DGS1', 'DGS5', 'DGS10']

In [22]:
NUMERICAL = GROWTH + TECHNICAL_INDICATORS + TECHNICAL_PATTERNS + CUSTOM_NUMERICAL + MACRO

In [23]:
# CHECK: NO OTHER INDICATORS LEFT
OTHER = [k for k in df_full.keys() if k not in OHLCV + CATEGORICAL + NUMERICAL + TO_DROP]
OTHER

['Close_x',
 'Dividends',
 'Stock Splits',
 'growth_future_30d',
 'is_positive_growth_30d_future',
 'Close_y']

In [24]:
df_full.Ticker.nunique()

33

In [25]:
# tickers, min-max date, count of daily observations
df_full.groupby(['Ticker'])['Date'].agg(['min','max','count'])

,min,max,count
Ticker,,,
AAPL,1980-12-12,2025-06-13,11217
ACN,2001-07-19,2025-06-13,6012
AMZN,1997-05-15,2025-06-13,7065
ASML,1995-03-15,2025-06-13,7614
AVGO,2009-08-06,2025-06-13,3989
BHARTIARTL.NS,2002-07-01,2025-06-13,5698
BRK-B,1996-05-09,2025-06-13,7322
CDI.PA,1992-01-27,2025-06-13,8610
GOOG,2004-08-19,2025-06-13,5239


In [26]:
# truncated df_full with 25 years of data (and defined growth variables)
df = df_full[df_full.Date>='2000-01-01']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191795 entries, 3490 to 5700
Columns: 204 entries, Open to ln_volume
dtypes: datetime64[ns](3), float64(130), int32(64), int64(5), object(2)
memory usage: 253.1+ MB


In [27]:
# let look at the features count and df size:
df[NUMERICAL].info()

<class 'pandas.core.frame.DataFrame'>
Index: 191795 entries, 3490 to 5700
Columns: 184 entries, growth_1d to DGS10
dtypes: float64(121), int32(62), int64(1)
memory usage: 225.3 MB


## 0.2) [Code snippet 1] Generating dummies

In [28]:
# what are the categorical features?
CATEGORICAL

['Month', 'Weekday', 'Ticker', 'ticker_type']

In [29]:
# dummy variables are not generated from Date and numeric variables
df.loc[:,'Month'] = df.Month.dt.strftime('%B')
df.loc[:,'Weekday'] = df.Weekday.astype(str)

/tmp/ipython-input-29-43129475.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['January' 'January' 'January' ... 'June' 'June' 'June']' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.loc[:,'Month'] = df.Month.dt.strftime('%B')
/tmp/ipython-input-29-43129475.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '1' '2' ... '2' '3' '4']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.loc[:,'Weekday'] = df.Weekday.astype(str)


In [30]:
# Generate dummy variables (no need for bool, let's have int32 instead)
dummy_variables = pd.get_dummies(df[CATEGORICAL], dtype='int32')

In [31]:
# TODO 1: define more categorical features, e.g. all combinations for <September+weekday>  (you'll see that September is actually an important dummy in one of the models)

In [32]:
dummy_variables.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191795 entries, 3490 to 5700
Data columns (total 55 columns):
 #   Column                Non-Null Count   Dtype
---  ------                --------------   -----
 0   Month_April           191795 non-null  int32
 1   Month_August          191795 non-null  int32
 2   Month_December        191795 non-null  int32
 3   Month_February        191795 non-null  int32
 4   Month_January         191795 non-null  int32
 5   Month_July            191795 non-null  int32
 6   Month_June            191795 non-null  int32
 7   Month_March           191795 non-null  int32
 8   Month_May             191795 non-null  int32
 9   Month_November        191795 non-null  int32
 10  Month_October         191795 non-null  int32
 11  Month_September       191795 non-null  int32
 12  Weekday_0             191795 non-null  int32
 13  Weekday_1             191795 non-null  int32
 14  Weekday_2             191795 non-null  int32
 15  Weekday_3             191795 non-null 

In [33]:
# get dummies names in a list
DUMMIES = dummy_variables.keys().to_list()

In [34]:
DUMMIES

['Month_April',
 'Month_August',
 'Month_December',
 'Month_February',
 'Month_January',
 'Month_July',
 'Month_June',
 'Month_March',
 'Month_May',
 'Month_November',
 'Month_October',
 'Month_September',
 'Weekday_0',
 'Weekday_1',
 'Weekday_2',
 'Weekday_3',
 'Weekday_4',
 'Weekday_5',
 'Weekday_6',
 'Ticker_AAPL',
 'Ticker_ACN',
 'Ticker_AMZN',
 'Ticker_ASML',
 'Ticker_AVGO',
 'Ticker_BHARTIARTL.NS',
 'Ticker_BRK-B',
 'Ticker_CDI.PA',
 'Ticker_GOOG',
 'Ticker_HDB',
 'Ticker_HINDUNILVR.NS',
 'Ticker_IBN',
 'Ticker_IDEXY',
 'Ticker_INFY',
 'Ticker_ITC.NS',
 'Ticker_JPM',
 'Ticker_LICI.NS',
 'Ticker_LLY',
 'Ticker_LT.NS',
 'Ticker_MC.PA',
 'Ticker_META',
 'Ticker_MSFT',
 'Ticker_NVDA',
 'Ticker_NVO',
 'Ticker_OR.PA',
 'Ticker_RELIANCE.NS',
 'Ticker_RMS.PA',
 'Ticker_SAP',
 'Ticker_SBIN.NS',
 'Ticker_SIE.DE',
 'Ticker_TCS.NS',
 'Ticker_TTE',
 'Ticker_V',
 'ticker_type_EU',
 'ticker_type_INDIA',
 'ticker_type_US']

In [35]:
# Concatenate the dummy variables with the original DataFrame
df_with_dummies = pd.concat([df, dummy_variables], axis=1)

In [36]:
df_with_dummies[NUMERICAL+DUMMIES].info()

<class 'pandas.core.frame.DataFrame'>
Index: 191795 entries, 3490 to 5700
Columns: 239 entries, growth_1d to ticker_type_US
dtypes: float64(121), int32(117), int64(1)
memory usage: 265.6 MB


## 0.3) [Code Snippet 2] Correlation analysis
* first approximation of "important" variables correlated with all variables we want to predict (TO_PREDICT)

In [37]:
TO_PREDICT

['growth_future_30d', 'is_positive_growth_30d_future']

In [38]:
corr_is_positive_growth_30d_future = df_with_dummies[NUMERICAL+DUMMIES+TO_PREDICT].corr()['is_positive_growth_30d_future']

In [39]:
# create a dataframe for an easy way to sort
corr_is_positive_growth_30d_future_df = pd.DataFrame(corr_is_positive_growth_30d_future)

In [40]:
corr_is_positive_growth_30d_future_df.sort_values(by='is_positive_growth_30d_future').head(5)

,is_positive_growth_30d_future
DGS10,-0.067204
DGS5,-0.059812
gdppot_us_yoy,-0.058374
gdppot_us_qoq,-0.058125
growth_brent_oil_365d,-0.056158


In [41]:
corr_is_positive_growth_30d_future_df.sort_values(by='is_positive_growth_30d_future').tail(8)

,is_positive_growth_30d_future
growth_btc_usd_7d,0.028577
Month_November,0.033807
Month_October,0.035410
growth_future_30d,0.696468
is_positive_growth_30d_future,1.000000
cdl3starsinsouth,NaN
cdlconcealbabyswall,NaN
cdlmathold,NaN


In [42]:
corr_growth_future_30d = df_with_dummies[NUMERICAL+DUMMIES+TO_PREDICT].corr()['growth_future_30d']

In [43]:
corr_growth_future_30d_df = pd.DataFrame(corr_growth_future_30d)

In [44]:
corr_growth_future_30d_df.sort_values(by='growth_future_30d').head(5)

,growth_future_30d
growth_brent_oil_365d,-0.084665
growth_dji_365d,-0.075940
growth_dax_365d,-0.060016
growth_wti_oil_365d,-0.055917
growth_snp500_365d,-0.055443


In [45]:
corr_growth_future_30d_df.sort_values(by='growth_future_30d').tail(8)

,growth_future_30d
ln_volume,0.052015
Ticker_NVDA,0.052434
Month_October,0.054752
is_positive_growth_30d_future,0.696468
growth_future_30d,1.000000
cdl3starsinsouth,NaN
cdlconcealbabyswall,NaN
cdlmathold,NaN


## 0.4) [Code snippet 3] Temporal split of ~25 years of data (by date)

In [46]:
def temporal_split(df, min_date, max_date, train_prop=0.7, val_prop=0.15, test_prop=0.15):
    """
    Splits a DataFrame into three buckets based on the temporal order of the 'Date' column.

    Args:
        df (DataFrame): The DataFrame to split.
        min_date (str or Timestamp): Minimum date in the DataFrame.
        max_date (str or Timestamp): Maximum date in the DataFrame.
        train_prop (float): Proportion of data for training set (default: 0.6).
        val_prop (float): Proportion of data for validation set (default: 0.2).
        test_prop (float): Proportion of data for test set (default: 0.2).

    Returns:
        DataFrame: The input DataFrame with a new column 'split' indicating the split for each row.
    """
    # Define the date intervals
    train_end = min_date + pd.Timedelta(days=(max_date - min_date).days * train_prop)
    val_end = train_end + pd.Timedelta(days=(max_date - min_date).days * val_prop)

    # Assign split labels based on date ranges
    split_labels = []
    for date in df['Date']:
        if date <= train_end:
            split_labels.append('train')
        elif date <= val_end:
            split_labels.append('validation')
        else:
            split_labels.append('test')

    # Add 'split' column to the DataFrame
    df['split'] = split_labels

    return df

In [47]:
min_date_df = df_with_dummies.Date.min()
max_date_df = df_with_dummies.Date.max()

df_with_dummies = temporal_split(df_with_dummies,
                                 min_date = min_date_df,
                                 max_date = max_date_df)

In [48]:
df_with_dummies['split'].value_counts()/len(df_with_dummies)

,count
split,
train,0.676399
test,0.163758
validation,0.159843


In [49]:
# remove the "segmentation" problem (warning message on df performance after many joins and data transformations)
new_df = df_with_dummies.copy()

# Q1: Dummies for Month and Week-of-Month

In [57]:
new_df['Month_WoM']=new_df['Month']+'_w'+(new_df['Date'].apply(lambda d: (d.day - 1) // 7 + 1)).astype(str)

In [58]:
new_dummy_variables = pd.get_dummies(new_df['Month_WoM'], dtype='int32')

In [59]:
new_dummy_variables.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191795 entries, 3490 to 5700
Data columns (total 60 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   April_w1      191795 non-null  int32
 1   April_w2      191795 non-null  int32
 2   April_w3      191795 non-null  int32
 3   April_w4      191795 non-null  int32
 4   April_w5      191795 non-null  int32
 5   August_w1     191795 non-null  int32
 6   August_w2     191795 non-null  int32
 7   August_w3     191795 non-null  int32
 8   August_w4     191795 non-null  int32
 9   August_w5     191795 non-null  int32
 10  December_w1   191795 non-null  int32
 11  December_w2   191795 non-null  int32
 12  December_w3   191795 non-null  int32
 13  December_w4   191795 non-null  int32
 14  December_w5   191795 non-null  int32
 15  February_w1   191795 non-null  int32
 16  February_w2   191795 non-null  int32
 17  February_w3   191795 non-null  int32
 18  February_w4   191795 non-null  int32
 19  Februa

In [60]:
NEW_DUMMIES = new_dummy_variables.keys().to_list()

In [62]:
# Concatenate the dummy variables with the original DataFrame
new_df_with_dummies = pd.concat([new_df, new_dummy_variables], axis=1)

In [64]:
corr_is_positive_growth_30d_future = new_df_with_dummies[NEW_DUMMIES+TO_PREDICT].corr()['is_positive_growth_30d_future']

In [65]:
corr_is_positive_growth_30d_future_df = pd.DataFrame(corr_is_positive_growth_30d_future)

In [68]:
corr_is_positive_growth_30d_future_df['abs_corr'] = corr_is_positive_growth_30d_future_df['is_positive_growth_30d_future'].abs()

In [71]:
corr_is_positive_growth_30d_future_df.sort_values(by='abs_corr', ascending=False).head(5)

,is_positive_growth_30d_future,abs_corr
is_positive_growth_30d_future,1.000000,1.000000
growth_future_30d,0.696468,0.696468
October_w4,0.024968,0.024968
November_w3,0.022097,0.022097
November_w2,0.018822,0.018822


# Q2: Define New "Hand" Rules on Macro and Technical Indicator Variables

In [72]:
for i in new_df_with_dummies.keys():
  print(i)

Open
High
Low
Close_x
Volume
Dividends
Stock Splits
Ticker
Year
Month
Weekday
Date
growth_1d
growth_3d
growth_7d
growth_30d
growth_90d
growth_365d
growth_future_30d
SMA10
SMA20
growing_moving_average
high_minus_low_relative
volatility
is_positive_growth_30d_future
ticker_type
index_x
adx
adxr
apo
aroon_1
aroon_2
aroonosc
bop
cci
cmo
dx
macd
macdsignal
macdhist
macd_ext
macdsignal_ext
macdhist_ext
macd_fix
macdsignal_fix
macdhist_fix
mfi
minus_di
mom
plus_di
dm
ppo
roc
rocp
rocr
rocr100
rsi
slowk
slowd
fastk
fastd
fastk_rsi
fastd_rsi
trix
ultosc
willr
index_y
ad
adosc
obv
atr
natr
ht_dcperiod
ht_dcphase
ht_phasor_inphase
ht_phasor_quadrature
ht_sine_sine
ht_sine_leadsine
ht_trendmod
avgprice
medprice
typprice
wclprice
index
cdl2crows
cdl3blackrows
cdl3inside
cdl3linestrike
cdl3outside
cdl3starsinsouth
cdl3whitesoldiers
cdlabandonedbaby
cdladvancedblock
cdlbelthold
cdlbreakaway
cdlclosingmarubozu
cdlconcealbabyswall
cdlcounterattack
cdldarkcloudcover
cdldoji
cdldojistar
cdldragonflydoji
